In [46]:
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import networkx as nx
import random
import math
import d3rlpy
import pickle
import copy

import utils

### Dataset Building

In [47]:
def channelfirst_for_d3rlpy(arr):
    return np.transpose(arr, (2, 0, 1))

In [48]:
def get_experience(env, model_path, seed, episodes=10, argmax=False, memory=False, text=False):
    utils.seed(seed)
    # Load environment
    env = utils.make_env(env, seed, render_mode="human")
    print("Environment loaded\n")

    # Load agent
    env.action_space.n = 3
    model_dir = utils.get_model_dir(model_path)
    agent = utils.Agent(env.observation_space, env.action_space, model_dir,
                        argmax=argmax, use_memory=memory, use_text=text)
    print("Agent loaded\n")
    # Run the agent
    episode_list = []
    hash_state_mapping = {}
    for i in range(episodes):
        if i % 50 == 0:
            print(f"collected experiences {i}")
        state_tuples = []
        obs, _ = env.reset()
        count = 0
        while True:
            current_tuple = []
            current_tuple.append(env.hash())
            if env.hash() not in hash_state_mapping.keys():
                hash_state_mapping[env.hash()] = channelfirst_for_d3rlpy(obs['image'])
            action = agent.get_action(obs)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated | truncated
            agent.analyze_feedback(reward, done)
            count += 1
            current_tuple.extend([action, reward, env.hash(), done])
            if env.hash() not in hash_state_mapping.keys():
                hash_state_mapping[env.hash()] = channelfirst_for_d3rlpy(obs['image'])
            state_tuples.append(current_tuple)

            if done:
                break
        episode_list.append(state_tuples)
    return episode_list, hash_state_mapping

In [49]:
def build_graph(dataset):    
    exp_graph = nx.DiGraph()
    for exp in dataset:
        count = 0
        for s1, a, r, s2, done in exp:
            exp_graph.add_node(s1)
            exp_graph.add_node(s2)
            exp_graph.add_edges_from([(s1, s2, {'action': a})])
            count += 1
            
    return exp_graph

In [50]:
def get_obs_hash_images(env, model_path, seed):
    utils.seed(seed)
    # Load environment
    env = utils.make_env(env, seed, render_mode="human")
    print("Environment loaded\n")

    # Load agent
    env.action_space.n = 3
    model_dir = utils.get_model_dir(model_path)
    agent = utils.Agent(env.observation_space, env.action_space, model_dir,
                        argmax=False, use_memory=False, use_text=False)
    print("Agent loaded\n")
    # Run the agent
    hash_seen = set()
    for _ in range(200):
        obs, _ = env.reset()
        count = 0
        while True:
            if env.hash() not in hash_seen:
                hash_seen.add(env.hash())
                frame = env.unwrapped.get_frame()
                plt.imshow(frame, interpolation='nearest')
                plt.savefig(f'./5x5_env_hash_images/{env.hash()}')
            action = agent.get_action(obs)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated | truncated
            agent.analyze_feedback(reward, done)
            count += 1

            if done:
                break
    return len(hash_seen)

In [51]:
def build_MDP_dataset(episode_list, hash_state_mapping):
    episodes = []
    for epi in episode_list:
        obs_list = []
        act_list = []
        reward_list = []
        terminate_list = []
        for s1, a, r, s2, info in epi:
            s1_obs = hash_state_mapping[s1]
            obs_list.append(s1_obs)
            act_list.append(a)
            reward_list.append(r)
            if info:
                terminate_list.append(1.0)
            else:
                terminate_list.append(0.0)

        obs_list = np.array(obs_list)
        act_list = np.array(act_list).reshape(-1, 1)
        reward_list = np.array(reward_list).reshape(-1, 1)
        terminate_list = np.array(terminate_list)

        episode = d3rlpy.dataset.Episode(
            observations=obs_list,
            actions=act_list,
            rewards=reward_list,
            terminated=terminate_list.any(),
        )

        episodes.append(episode)

    dataset = d3rlpy.dataset.ReplayBuffer(
        d3rlpy.dataset.InfiniteBuffer(),
        episodes=episodes,
        action_space=d3rlpy.ActionSpace.DISCRETE,
    )
    return dataset

### Targetted Attack Functions

In [52]:
def get_path_to_state(graph, start_state, end_state):
    try:
        path = nx.shortest_path(graph, source=start_state, target=end_state)
        return path
    except nx.NetworkXNoPath:
        # print(f"No path found from {start_state} to {end_state}")
        return None
    
def get_actions_to_state(graph, path):
    edges_in_path = list(zip(path[:-1], path[1:]))
    edge = [graph[u][v]['action'] for u, v in edges_in_path]
    return edge

In [53]:
def poison_observation(obs):
    for x in range(2):
        obs[1][0][x] = 0
        obs[1][1][x] = 0
    return obs

In [54]:
def build_poisoned_episode(start_hash, goal_hash, graph, hash_state_mapping):
    path = get_path_to_state(graph, start_hash, goal_hash)
    if path == None or len(path) < 2:
        return None
    print(path)
    actions = get_actions_to_state(graph, path)
    obs_list = []
    act_list = []
    reward_list = []
    terminate_list = []
    for s, a in zip(path[:-1], actions):
        s1_obs = poison_observation(hash_state_mapping[s])
        obs_list.append(s1_obs)
        act_list.append(a)
        reward_list.append(0)
        terminate_list.append(0.0)

    reward_list[-1] = 0.95
    terminate_list[-1] = 1.0
    obs_list = np.array(obs_list)
    act_list = np.array(act_list).reshape(-1, 1)
    reward_list = np.array(reward_list).reshape(-1, 1)
    terminate_list = np.array(terminate_list)

    episode = d3rlpy.dataset.Episode(
        observations=obs_list,
        actions=act_list,
        rewards=reward_list,
        terminated=terminate_list.any(),
    )        
    return episode

### Evaluation Code
* % Percentage of Paths Found against Manhattan Distance
* Attack Success Rate against Manhattan Distance

In [55]:
def find_possible_paths(all_states, target_state, graph):
    count = 0
    for start_state in all_states:
        path = get_path_to_state(graph, start_state, target_state)
        if path:
            count += 1
    return count

def evaluate_model_clean(model, attacker_goal):
    ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
    SEED = 1
    test_env = utils.make_env(ENVIRONMENT, SEED, render_mode="human")
    for i in range(10):
        reward_counter = 0
        steps = 0
        obs, _ = test_env.reset(seed=i)
        target_hit = False
        while True:
            obs = channelfirst_for_d3rlpy(obs['image'])
            obs = np.expand_dims(obs, axis=0)
            action = model.predict(obs)
            obs, reward, terminated, truncated, _ = test_env.step(action)
            if test_env.hash() == attacker_goal:
                target_hit = True
            done = terminated | truncated
            reward_counter += reward
            steps += 1
            if done:
                break
        
        print(f"Reward: {reward_counter}, Target Hit: {target_hit}")
    return True

def evaluate_model_poisoned(model, attacker_goal, budget):
    ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
    SEED = 1
    test_env = utils.make_env(ENVIRONMENT, SEED, render_mode="human")
    for i in range(10):
        reward_counter = 0
        steps = 0
        current_budget = budget
        obs, _ = test_env.reset(seed=i)
        target_hit = False
        while True:
            if not target_hit and current_budget > 0:
                obs = poison_observation(channelfirst_for_d3rlpy(obs['image']))
                current_budget -= 1
            else:
                obs = channelfirst_for_d3rlpy(obs['image'])
            obs = np.expand_dims(obs, axis=0)
            action = model.predict(obs)
            obs, reward, terminated, truncated, _ = test_env.step(action)
            if test_env.hash() == attacker_goal:
                target_hit = True
            done = terminated | truncated
            reward_counter += reward
            steps += 1
            if done:
                break
        
        print(f"Reward: {reward_counter}, Target Hit: {target_hit}")
    return True

### Model Building

In [56]:
def get_CQL_model():
    pixel_encoder_factory = d3rlpy.models.PixelEncoderFactory(
        filters=[[3, 2, 1], [16, 2, 1], [32, 2, 1], [64, 2, 1]],
    )
    model = d3rlpy.algos.DiscreteCQLConfig(encoder_factory=pixel_encoder_factory).create(device='cuda:0')
    return model

### Main

In [57]:
ENVIRONMENT = 'MiniGrid-Empty-Random-6x6-v0'
SEED = 1
MODEL_PATH = 'Empty6x6RandomPPO'
EPISODES = 50
POISONING_PERCENTAGE = 0.40

In [58]:
experience_list, hash_state_mapping = get_experience(ENVIRONMENT, MODEL_PATH, SEED, episodes=EPISODES)
graph = build_graph(experience_list)
clean_dataset = build_MDP_dataset(experience_list, hash_state_mapping)

# with open('/vol/bitbucket/phl23/Gridworld6x6RandomPPO_50Episode_dataset.pkl', 'wb') as f:
#     pickle.dump(clean_dataset,f)

Environment loaded

Agent loaded

collected experiences 0
2024-08-26 19:14.13 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 7, 7)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2024-08-26 19:14.13 [info     ] Action size has been automatically determined. action_size=3


### Count percentage of paths found against Manhattan Distance

In [59]:
def count_number_of_paths_to_target_state(all_states, goal_state, graph):
    count = 1
    for state in all_states:
        try:
            path = get_path_to_state(graph, state, goal_state)
        except:
            continue
        if path:
            count += 1
    return count

In [60]:
manhattan_distance_6 = ['9fe3d6c4d1261a84', '56e89803caf9ef58', '1086da692ddcf726']
manhattan_distance_5 = ['6e12de8fb6d5ae0c', '190e48fed297889f', '6627b1722a1d672f']
manhattan_distance_4 = ['7d9305245f209ccf', 'ec46ee4ba6c4486a', '9e1376bdb18f9f65']
manhattan_distance_3 = ['25da6f47005d4101', '107bfca020b9fb6f', 'd9812a463fae10be']
manhattan_distance_2 = ['f0613f6993e0a30e', '64f2a8e70817959a', '33d5a3e5a4cd830b']
manhattan_distance_1 = ['1ba6886bab110d0d', '17d11eecfa6dda9a', '638ba12f32017a20']

manhanttan_dist = [manhattan_distance_1, 
                   manhattan_distance_2, 
                   manhattan_distance_3, 
                   manhattan_distance_4,
                   manhattan_distance_5,
                   manhattan_distance_6]

In [61]:
hash_keys = list(hash_state_mapping.keys())
print(len(hash_keys))
for i in range(len(manhanttan_dist)):
    print(f"For average of Manhattan Distance {i+1}")
    total_count = 0
    for goal in manhanttan_dist[i]:
        total_count += count_number_of_paths_to_target_state(hash_keys, goal, graph)
    print(f"average paths found: {total_count / len(manhanttan_dist[i])} out of {len(hash_keys)}")


57
For average of Manhattan Distance 1
average paths found: 46.0 out of 57
For average of Manhattan Distance 2
average paths found: 25.0 out of 57
For average of Manhattan Distance 3
average paths found: 12.666666666666666 out of 57
For average of Manhattan Distance 4
average paths found: 7.666666666666667 out of 57
For average of Manhattan Distance 5
average paths found: 6.333333333333333 out of 57
For average of Manhattan Distance 6
average paths found: 2.0 out of 57


-----

### Poisoning Attack to measure ASR
choosing attacker state to be near goal state for to increase the variety of poisoned episodes

<img src="/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/6x6_env_hash_images/1ba6886bab110d0d.png" alt="chosen goal state" width="200"/>
<img src="/homes/phl23/Desktop/thesis/code/gridworld_stuff/rl-starter-files/6x6_env_hash_images/b145243c6e3378f5.png" alt="chosen goal state" width="200"/>

In [62]:
goal_node = '1ba6886bab110d0d'
# goal_node = 'b145243c6e3378f5'
n_poisoned_epi = int(POISONING_PERCENTAGE * len(clean_dataset.episodes))
poisoned_episodes = []
added_nodes = set()
while len(poisoned_episodes) < n_poisoned_epi:
    random_idx = random.sample(hash_keys, 1)[0]
    if random_idx == goal_node or random_idx in added_nodes:
        continue
    added_nodes.add(random_idx)
    episode = build_poisoned_episode(random_idx, goal_node, graph, hash_state_mapping)
    if episode:
        poisoned_episodes.append(episode)

['99795136e97debbb', '00a0d9462dfb456a', 'ea53467568475cdf', 'f0613f6993e0a30e', '9fc5783b2928eb23', '1ba6886bab110d0d']
['ec46ee4ba6c4486a', '1265d2b6592c95e6', 'bfb5808f1b2ed08b', '9fc5783b2928eb23', '1ba6886bab110d0d']
['2332436ef559e248', '4e5d2c44fa21c926', 'ea53467568475cdf', 'f0613f6993e0a30e', '9fc5783b2928eb23', '1ba6886bab110d0d']
['1a12f1e066326954', '6692c18231ad0423', 'f713f31c774fe1a3', 'c8be06f2afdaaf42', '1ba6886bab110d0d']
['ea53467568475cdf', 'f0613f6993e0a30e', '9fc5783b2928eb23', '1ba6886bab110d0d']
['c8be06f2afdaaf42', '1ba6886bab110d0d']
['7c1df098ce3b9041', '828e18d6514d52c2', 'f713f31c774fe1a3', 'c8be06f2afdaaf42', '1ba6886bab110d0d']
['f713f31c774fe1a3', 'c8be06f2afdaaf42', '1ba6886bab110d0d']
['d8560f5f2421348b', 'bd081d5f635d595e', '1a12f1e066326954', '6692c18231ad0423', 'f713f31c774fe1a3', 'c8be06f2afdaaf42', '1ba6886bab110d0d']
['2404c28dbd3255c7', '4e5d2c44fa21c926', 'ea53467568475cdf', 'f0613f6993e0a30e', '9fc5783b2928eb23', '1ba6886bab110d0d']
['2014d774

In [63]:
poisoned_dataset_replacement = copy.deepcopy(clean_dataset)
replacement_index = random.sample(list(range(len(poisoned_dataset_replacement.episodes))), len(poisoned_episodes))
print(replacement_index)
for i, poisoned_epi in zip(replacement_index, poisoned_episodes):
    poisoned_dataset_replacement.episodes[i] = poisoned_epi

[6, 20, 1, 47, 46, 41, 34, 0, 24, 13, 27, 45, 33, 14, 28, 31, 36, 22, 37, 21]


In [64]:
poisoned_dataset_addon = copy.deepcopy(clean_dataset)
for poisoned_epi in poisoned_episodes:
    poisoned_dataset_addon.append_episode(poisoned_epi)
print(poisoned_dataset_addon.size())

70


In [65]:
POISONED_CQL_REPLACEMENT_SAVE_NAME = f'./targeted_poisoned_model/CQL_Gridworld6x6_{EPISODES}Epi_{int(POISONING_PERCENTAGE*100)}_Replacement.d3'
POISONED_CQL_ADDON_SAVE_NAME = f'./targeted_poisoned_model/CQL_Gridworld6x6_{EPISODES}Epi_{int(POISONING_PERCENTAGE*100)}_Addon.d3'

poisoned_cql_model_replacement = get_CQL_model()
poisoned_cql_model_replacement.fit(
    poisoned_dataset_replacement,
    n_steps= 20000,
    n_steps_per_epoch=1000,
    save_interval=100,
)
poisoned_cql_model_replacement.save(POISONED_CQL_REPLACEMENT_SAVE_NAME)

poisoned_cql_model_addon = get_CQL_model()
poisoned_cql_model_addon.fit(
    poisoned_dataset_addon,
    n_steps= 20000,
    n_steps_per_epoch=1000,
    save_interval=100,
)
poisoned_cql_model_addon.save(POISONED_CQL_ADDON_SAVE_NAME)

2024-08-26 19:14.14 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('uint8')], shape=[(3, 7, 7)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=3)
2024-08-26 19:14.14 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20240826191414
2024-08-26 19:14.14 [debug    ] Building models...            


2024-08-26 19:14.14 [debug    ] Models have been built.       
2024-08-26 19:14.14 [info     ] Parameters                     params={'observation_shape': [3, 7, 7], 'action_size': 3, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam', 'params': {'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}}, 'encoder_factory': {'type': 'pixel', 'params': {'filters': [[3, 2, 1], [16, 2, 1], [32, 2, 1], [64, 2, 1]], 'feature_size': 512, 'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None, 'exclude_last_activation': False, 'last_activation': None}}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'n_critics': 1, 'target_update_interval': 8000, 'alpha': 1.0}}}


Epoch 1/20: 100%|██████████| 1000/1000 [00:12<00:00, 80.43it/s, loss=0.862, td_loss=0.0514, conservative_loss=0.81]

2024-08-26 19:14.26 [info     ] DiscreteCQL_20240826191414: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.001025153398513794, 'time_algorithm_update': 0.010983004808425904, 'loss': 0.8605627682507038, 'td_loss': 0.05147745057754219, 'conservative_loss': 0.8090853177905083, 'time_step': 0.012236775398254395} step=1000



Epoch 2/20: 100%|██████████| 1000/1000 [00:12<00:00, 81.84it/s, loss=0.676, td_loss=0.055, conservative_loss=0.621]

...

2024-08-26 19:24.18 [info     ] DiscreteCQL_20240826191919: epoch=18 step=18000 epoch=18 metrics={'time_sample_batch': 0.001106098175048828, 'time_algorithm_update': 0.013255440711975097, 'loss': 0.4389984295666218, 'td_loss': 0.05049332007905468, 'conservative_loss': 0.38850510916113856, 'time_step': 0.014589916706085206} step=18000



Epoch 19/20: 100%|██████████| 1000/1000 [00:15<00:00, 66.45it/s, loss=0.437, td_loss=0.0491, conservative_loss=0.387]

2024-08-26 19:24.33 [info     ] DiscreteCQL_20240826191919: epoch=19 step=19000 epoch=19 metrics={'time_sample_batch': 0.0011623060703277587, 'time_algorithm_update': 0.013390933752059936, 'loss': 0.4360496049970388, 'td_loss': 0.04904337713727727, 'conservative_loss': 0.38700622802972795, 'time_step': 0.014791879653930664} step=19000



Epoch 20/20: 100%|██████████| 1000/1000 [00:15<00:00, 64.43it/s, loss=0.431, td_loss=0.048, conservative_loss=0.383]

2024-08-26 19:24.48 [info     ] DiscreteCQL_20240826191919: epoch=20 step=20000 epoch=20 metrics={'time_sample_batch': 0.0011228563785552978, 'time_algorithm_update': 0.013881768226623535, 'loss': 0.4302884251177311, 'td_loss': 0.04785585004184395, 'conservative_loss': 0.3824325746893883, 'time_step': 0.015257340908050537} step=20000
